정규화를 해서 dt에 넣어보기 -> 모든 데이터를 0~1 사이의 값이 될 수 있또록
lead_time, avg_price_per_room -> cate ->  num으로 바꾸기


In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


In [3]:

train = pd.read_csv('sorted_lead_time.csv')
train.keys()

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status',
       'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
       'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
       'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
       'Complementary', 'Online', 'Offline', 'Aviation', 'lead_time_sqrt'],
      dtype='object')

In [8]:


'''
feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests',
                 'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
                 'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
                 'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
                 'Complementary', 'Online', 'Offline', 'Aviation']
'''


train = pd.read_csv('onehot_sample.csv')
feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'no_of_special_requests',
                 'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
                 'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
                 'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
                 'Complementary', 'Online', 'Offline', 'Aviation']
# 87 프로 피쳐
# feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
#                  'no_of_week_nights', 'required_car_parking_space',
#                  'lead_time', 'arrival_year', 'arrival_month',
#                  'arrival_date', 'repeated_guest',
#                  'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
#                  'avg_price_per_room', 'no_of_special_requests',
#                  'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
#                  'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
#                  'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
#                  'Complementary', 'Online', 'Offline', 'Aviation']

X = train[feature_names]
y = train["booking_status"]

# Decision Tree 학습/예측/평가
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Decision Tree 학습/예측/평가
dt_model = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 2, criterion="gini", min_samples_split = 2)
scores = cross_val_score(dt_model, X, y, cv = 10)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77322835 0.78976378 0.80385979 0.79243797 0.78141    0.78180386
 0.78141    0.78534856 0.78101615 0.78653013]
Average CV Score:  0.785680858915873
Number of CV Scores used in Average:  10


In [6]:

def get_dt_graph(dt_classifier):
    fig = plt.figure(figsize=(25, 20))
    _ = dt_model.plot_tree(dt_classifier,
                       feature_names=X.columns,
                       class_names=['No Disease', "Disease"],
                       filled=True)


def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(
        y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(
        y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))


params = {
    'max_depth' : [2, 4, 6, 8, 10], 
    'min_samples_leaf' : [2, 4, 6, 8, 10], 
    'criterion' : ["gini", "entropy"],
    'min_samples_split' : [2, 4, 6, 8, 10]
}


# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dt_model,
                           param_grid=params,
                           cv=10, n_jobs=-1, verbose=1, scoring="accuracy")

grid_search.fit(X_train, y_train)


Fitting 10 folds for each of 250 candidates, totalling 2500 fits


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(max_depth=10, min_samples_leaf=2),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [2, 4, 6, 8, 10],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             scoring='accuracy', verbose=1)

In [7]:
GridSearchCV(cv = 10, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_samples_split': [5, 10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

score_df = pd.DataFrame(grid_search.cv_results_)
# score_df.to_csv('dt_model_grid_search.csv')
score_df.nlargest(5, "mean_test_score")


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
102,0.121201,0.031225,0.004999,0.001964,gini,10,2,6,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.774803,...,0.788976,0.798319,0.787290,0.787290,0.771008,0.780462,0.791492,0.785129,0.008127,1
101,0.093257,0.005493,0.004662,0.001370,gini,10,2,4,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.775853,...,0.790551,0.797794,0.788866,0.786765,0.771008,0.780987,0.790966,0.785077,0.008038,2
100,0.113577,0.025269,0.004797,0.002854,gini,10,2,2,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.775853,...,0.790551,0.798319,0.788340,0.786239,0.771008,0.779937,0.790966,0.784867,0.007946,3
104,0.098486,0.009546,0.004114,0.000578,gini,10,2,10,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.775853,...,0.790551,0.796744,0.786239,0.787290,0.770483,0.778887,0.791492,0.784814,0.008015,4
114,0.083639,0.010655,0.004455,0.001527,gini,10,6,10,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.776378,...,0.794226,0.796218,0.782563,0.786239,0.770483,0.777836,0.792017,0.784656,0.008527,5


In [63]:

dt_model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=10, min_samples_leaf=2)

In [64]:
y_pred = dt_model.predict(X_test)
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
evaluate_model(dt_model)

Accuracy: 0.87
Train Accuracy : 0.8873660995589162
Train Confusion Matrix:
[[11961   873]
 [ 1272  4938]]
--------------------------------------------------
Test Accuracy : 0.8724007561436673
Test Confusion Matrix:
[[3959  321]
 [ 489 1579]]


In [24]:
from sklearn.preprocessing import OneHotEncoder

real_test = pd.read_csv('test.csv')

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['type_of_meal_plan']])
train_cat = pd.DataFrame(train_cat)
real_test['meal_type_1'] = train_cat.loc[:, 0]
real_test['meal_type_2'] = train_cat.loc[:, 1]
real_test['meal_type_3'] = train_cat.loc[:, 2]
real_test['meal_type_4'] = train_cat.loc[:, 3]

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['room_type_reserved']])
train_cat = pd.DataFrame(train_cat)
real_test['Room_Type_1'] = train_cat.loc[:, 0]
real_test['Room_Type_2'] = train_cat.loc[:, 1]
real_test['Room_Type_3'] = train_cat.loc[:, 2]
real_test['Room_Type_4'] = train_cat.loc[:, 3]
real_test['Room_Type_5'] = train_cat.loc[:, 4]
real_test['Room_Type_6'] = train_cat.loc[:, 5]
real_test['Room_Type_7'] = train_cat.loc[:, 6]

train_cat = ohe.fit_transform(real_test[['market_segment_type']])
train_cat = pd.DataFrame(train_cat)
real_test['Corporate'] = train_cat.loc[:, 0]
real_test['Complementary'] = train_cat.loc[:, 1]
real_test['Online'] = train_cat.loc[:, 2]
real_test['Offline'] = train_cat.loc[:, 3]
real_test['Aviation'] = train_cat.loc[:, 4]

real_test.to_csv('onehot_test.csv', index=False)



In [33]:
real_test = pd.read_csv('onehot_test.csv')

feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests', 'meal_type_1',
                 'meal_type_2', 'meal_type_3', 'meal_type_4', 'Room_Type_1',
                 'Room_Type_2', 'Room_Type_3', 'Room_Type_4', 'Room_Type_5',
                 'Room_Type_6', 'Room_Type_7', 'Corporate', 'Complementary', 'Online',
                 'Offline', 'Aviation']

test = real_test[feature_names]
y_pred = dt_model.predict(test)

sample = pd.DataFrame()
sample['Booking_ID'] = real_test['Booking_ID']
sample['booking_status'] = y_pred
sample.to_csv('sample.csv', index=False)
# real_test.keys()


c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Room_Type_1
- Room_Type_2
- Room_Type_3
- Room_Type_4
- Room_Type_5
- ...
Feature names seen at fit time, yet now missing:
- room_type_1
- room_type_2
- room_type_3
- room_type_4
- room_type_5
- ...

  warnings.warn(message, FutureWarning)
